USING AUTOLOADER FOR CUSTOMERS

In [0]:
spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion")

In [0]:
df = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("inferSchema", "true")
    .option(
        "cloudFiles.schemaLocation",
        "/Volumes/dea_course_catalog_gizmobox/landing/operational_data/customers_autoloader/_schema",
    )
    .option(
        "cloudFiles.inferColumnTypes", "true"
    )
    .option("cloudFiles.schemaHints","date_of_birth DATE, created_timestamp TIMESTAMP")
    .load(
        "/Volumes/dea_course_catalog_gizmobox/landing/operational_data/customers_autoloader/"
    )
)

In [0]:
from pyspark.sql.functions import current_timestamp,col

In [0]:
customers_df = df.withColumn("ingestion_date", current_timestamp()).withColumn(
    "filepath", col("_metadata.file_path")
)

In [0]:
streaming_query = (
    customers_df.writeStream.format("delta")
    .outputMode("append")
    .option(
        "checkpointLocation",
        "/Volumes/dea_course_catalog_gizmobox/landing/operational_data/customers_autoloader/checkpoint",
    )
    .toTable("dea_course_catalog_gizmobox.bronze.customers_autoloader")
)

In [0]:
%sql

select * from dea_course_catalog_gizmobox.bronze.customers_autoloader